In [1]:
import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling

input_raster = "input.tif"
output_raster = "reprojected_reclassified.tif"
target_epsg = "EPSG:4326"

with rasterio.open(input_raster) as src:
    # Calculate transformation for reprojection
    transform, width, height = calculate_default_transform(
        src.crs, target_epsg, src.width, src.height, *src.bounds
    )

    # Set up output metadata
    dst_meta = src.meta.copy()
    dst_meta.update({
        "crs": target_epsg,
        "transform": transform,
        "width": width,
        "height": height,
        "compress": "lzw",
        "nodata": 0
    })

    # Create the output raster
    with rasterio.open(output_raster, "w", **dst_meta) as dst:
        for band in range(1, src.count + 1):  # Loop through bands
            src_array = src.read(band)

            # Apply reclassification
            remapped_array = np.vectorize(lambda x: class_remap.get(x, x))(src_array)

            # Reproject & write
            reproject(
                source=remapped_array,
                destination=rasterio.band(dst, band),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_epsg,
                resampling=Resampling.nearest
            )

print("✅ Reprojection & reclassification complete:", output_raster)

RasterioIOError: input.tif: No such file or directory

In [ ]:
from rasterio.mask import mask
import geopandas as gpd
import rasterio as rio

m = gpd.read_file('/tmp/tmpznbidgvg/free_ukraine_20240522_32636_reprojected.shp')

with rio.open('/tmp/tmpznbidgvg/AO_20240708_c_20240716_WC_SC_fused.tif') as src:
    mask(src, m.geometry)

MemoryError: Unable to allocate 64.0 GiB for an array with shape (75835, 113335) and data type int64

In [ ]:
from raster_handling import merge_classes

merge_dict = {0:0,
                    1:0,
                    2:2,
                    3:3,
                    4:4,
                    5:2}

merge_classes('/tmp/tmpyeav88m2/AO_20240708_c_20240716_WC_SC_reprojected.tif', '/tmp/tmpyeav88m2/AO_20240708_c_20240716_WC_SC_reprojected_merged.tif', merge_dict)

Reclassing raster /tmp/tmpyeav88m2/AO_20240708_c_20240716_WC_SC_reprojected.tif
Reading raster


/home/rohan/.pyenv/versions/3.10.16/lib/python3.10/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [ ]:
from osgeo import gdal
import numpy as np
raster = gdal.Open('/tmp/tmpwhjg4x_h/AO_20240708_c_20240716_WC_SC_fused.tif')
band = raster.GetRasterBand(1)
arr = band.ReadAsArray()
print('read')
unique, counts = np.unique(arr, return_counts=True)
print(unique, counts)                                                                                                                           
print(unique, counts)

read


In [ ]:
import numpy as np

data = {np.uint8(0): np.int64(1867533550), np.uint8(2): np.int64(487860710), np.uint8(4): np.int64(1002773735), np.uint8(3): np.int64(64611936)}
transformed_data = {key: (value * 12 * 12) / (100 * 100) for key, value in data.items()}

# Print results
for key, value in transformed_data.items():
    print(f"{key}: {value}")

0: 26892483.12
2: 7025194.224
4: 14439941.784
3: 930411.8784


In [ ]:
import pandas as pd

df = pd.read_csv('/home/rohan/nasa-harvest/unbiases-area-estimator/AO_20240708_c_20240716_WC_SC_free_ukraine_20240522_32636_reprojected_samples.csv')

print(df['stratum_id'].value_counts())                                                                                                      

stratum_id
0    432
4    232
2    113
3     15
Name: count, dtype: int64


In [1]:
import pandas as pd
from unbiased_estimation import Region

df = pd.read_csv('/home/rohan/nasa-harvest/AnnotationProject/crop-maps/FullSample/outputfiles/AO_20230708_c_20240716_WC_SC_Free_4C_annot.csv')

predicted = df['strata']
true = df['class']

wh = {
    0: 0.545618695, # Non cultivated
    2: 0.142533504, # Winter and spring cereal
    3: 0.018877059, #rapeseed
    4: 0.292970741 # summer
}

areas = {
    0: 26892471.25,
    2: 7025195.81,
    3: 930413.09,
    4: 14439951.00,
}

res = Region.compute_area_metrics(true, predicted, wh, areas)
print(res)

   Class  Total Predicted  Total True  Total Positive  User's Accuracy  \
0      0            420.0       438.0           398.0         0.947619   
1      2            115.0       110.0            89.0         0.773913   
2      3             75.0        80.0            74.0         0.986667   
3      4            240.0       222.0           207.0         0.862500   

   Producer's Accuracy  User's Accuracy Proportional  \
0             0.908676                      0.947619   
1             0.809091                      0.773913   
2             0.925000                      0.986667   
3             0.932432                      0.862500   

   Producer's Accuracy Proportional  User's Variance  Producer's Variance  \
0                          0.913167         0.000118             0.000140   
1                          0.806778         0.001535             0.001184   
2                          0.713026         0.000178             0.006787   
3                          0.932607     